In [29]:
data_dir = '/mnt/lts/nfs_fs02/sadow_lts/personal/yusukemh/PI-CASC'
import pandas as pd
import numpy as np
# pd.read_csv(f"{data_dir}/processed_data/ml_dataset.csv")


In [30]:
df = pd.read_csv('/mnt/lts/nfs_fs02/sadow_lab/personal/yusukemh/PI-CASC/processed_data/ml_dataset.csv')
df

,skn,year,month,data_in,filled,name,lat,lon,air2m,air1000_500,...,skt_4,skt_5,slp_0,slp_1,slp_2,slp_3,slp_4,slp_5,elevation,season_wet
0,1.00,1948,1,3.200000,False,KALAE,18.916176,-155.674994,295.39603,31.299995,...,23.194658,23.052715,1014.175476,1013.945740,1014.084900,1015.194092,1014.699646,1014.406311,35.000000,1
1,1.00,1949,1,8.700000,False,KALAE,18.916176,-155.674994,295.52832,32.030000,...,23.426136,23.062620,1014.218628,1014.555359,1015.127014,1015.122925,1015.357971,1015.803894,35.000000,1
2,1.00,1950,1,5.140000,True,KALAE,18.916176,-155.674994,295.88953,31.689995,...,23.475718,23.184557,1012.599609,1013.097717,1013.643860,1012.946472,1013.287842,1013.578979,35.000000,1
3,1.00,1951,1,4.280000,True,KALAE,18.916176,-155.674994,295.64987,31.349998,...,23.691616,23.522514,1016.933472,1017.046753,1017.619690,1018.097595,1018.057739,1018.303101,35.000000,1
4,1.00,1952,1,8.230000,True,KALAE,18.916176,-155.674994,295.32180,29.190002,...,23.605747,23.200550,1014.928284,1015.308533,1016.047974,1016.068726,1016.284241,1016.759949,35.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865532,425.50,2012,11,3.791339,False,HAIKU 3.2 SE HI US,20.876100,-156.304700,296.95737,31.419998,...,25.403799,24.937208,1015.779602,1016.139221,1016.532898,1017.147522,1017.136963,1017.056274,1371.105564,1
865533,425.50,2012,12,6.913386,False,HAIKU 3.2 SE HI US,20.876100,-156.304700,296.24292,31.040000,...,24.352453,24.012741,1015.605713,1016.248657,1016.884888,1016.964722,1017.289978,1017.555420,1371.105564,1
865534,425.50,2012,10,0.381890,False,HAIKU 3.2 SE HI US,20.876100,-156.304700,297.81000,31.590004,...,25.573721,25.196182,1015.071716,1015.309692,1015.714539,1015.955383,1015.960449,1016.076050,1371.105564,0
865535,324.30,1949,12,1.500000,False,KULA,20.740136,-156.317187,296.32840,32.189995,...,24.189648,23.809095,1014.235779,1014.495178,1015.020569,1015.382690,1015.547363,1015.869629,3650.000000,1


In [15]:
(df.query("skn==79")['filled'] == False).sum()

758

In [26]:
def assign_fold(df, n_folds=5, min_data=None, nonfilled=False):
    if nonfilled: # if nonfilled == True, remove any row with filled == True
        df = df[df['filled'] != True]
    df_n_data = df.groupby('skn').size().reset_index().rename(columns={0: "n_data"})
    if min_data is not None:
        df_n_data = df_n_data[df_n_data['n_data'] > min_data]
    df = df.merge(df_n_data, left_on='skn', right_on='skn', how='inner')
    
    df_len_by_month = pd.DataFrame(df.groupby(by=['year', 'month']).size()).reset_index().rename({0: "len"}, axis=1)
    df_len_by_month['cumsum'] = df_len_by_month['len'].cumsum()
    n_samples_total = df_len_by_month['cumsum'].iloc[-1]
    n_samples_per_fold = np.ceil(n_samples_total / n_folds)
    
    df_len_by_month['fold'] = df_len_by_month.apply(lambda row: int(row['cumsum'] / n_samples_per_fold), axis=1)
    
    df_w_fold = pd.merge(left=df, right=df_len_by_month, left_on=['year', 'month'], right_on=['year', 'month'])
    
    return df_w_fold

In [27]:
df_fold = assign_fold(df, min_data=750, nonfilled=True)

In [28]:
for fold in range(5):
    print(df_fold.query(f"fold=={fold}").shape)

(3673, 126)
(3703, 126)
(3686, 126)
(3700, 126)
(3709, 126)
